In [1]:
import urllib
import requests
import re
import time
from bs4 import BeautifulSoup
from heapq import * 
import hashlib
import threading
from pyhanlp import *
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('enable-automation')
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--dns-prefetch-disable')
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.set_page_load_timeout(2)

max_num=60
head = {        'Connection': 'Keep-Alive',
        'Accept': 'text/html, application/xhtml+xml, */*',
        'Accept-Language': 'en-US,en;q=0.8,zh-Hans-CN;q=0.5,zh-Hans;q=0.3',
        'Accept-Encoding': 'gzip, deflate',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
       }

def baidu_find(url):
    #0即深网 1即明网 -1即被封
    keyword=urllib.parse.quote(url)
    url2=r'https://www.baidu.com/s?tn=02003390_14_hao_pg&ie=UTF-8&wd=site%3A'+keyword+r'&rsv_spt=1&rsv_iqid=0xdb5b72850006626d&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&tn=02003390_14_hao_pg&rsv_enter=0&rsv_dl=tb&rsv_sug3=6&rsv_sug1=6&rsv_sug7=101&rsv_btype=i&prefixsug=sdasd&rsp=0&inputT=1704&rsv_sug4=4371'
    res=requests.get(url2,headers=head)
    res.encoding='utf-8'

    flag1=len(re.findall(r'很抱歉，没有找到',res.text))
    flag2=len(re.findall(r'<title>百度安全验证</title>',res.text))
    if flag2>0:
        return -1 #被封了,找个屁
    elif flag1>0:
        return 0 #没被封，没找到
    else:
        return 1 #没被封，找到了

def md5_(s):
    m1 = hashlib.md5()
    m1.update(s.encode("utf-8"))
    token = m1.hexdigest()
    return token[0:32:5]
mode=re.compile(r'http[s]?://(.*)')
begin_t=time.time()
md5s=[0 for x in range(16**7)]
print(time.time()-begin_t)

c:\users\zita\miniconda2\envs\python35\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use options instead of chrome_options


18.027283668518066


In [2]:
import collections
waiting_list= collections.deque()

heap = [] #for 更新事件 每次取出预定时间最小的

##        
class url_class:
    def __init__(self,url=""):
        self.rank=int(4*len(re.findall(r'/',url))+1*len(re.findall('\?',url))+1*len(re.findall('=',url)) \
        +1*len(re.findall(r'#',url))-3*len(re.findall(r'index',url))) #rank越大越不重要
        self.url=url
    def __lt__(self, other):
        return self.rank < other.rank
    def __gt__(self, other):
        return self.rank > other.rank
    def __eq__(self,other):
        return self.rank == other.rank
    def __ne__(self,other):
        return self.rank != other.rank
    def __le__(self,other):
        return self.rank <= other.rank
    def __ge__(self,other):
        return self.rank >= other.rank
    def __cmp__(self,other):
        return cmp(self.rank,other.rank)
##广度优先套优先级

class update_event:
    def __init__(self,deltat=30,url=""):
        self.set_time=time.time()+deltat
        self.url=url_class(url)
    def __lt__(self, other):
        return self.set_time < other.set_time
    def __gt__(self, other):
        return self.set_time > other.set_time
    def __eq__(self,other):
        return self.set_time == other.set_time
    def __ne__(self,other):
        return self.set_time != other.set_time
    def __le__(self,other):
        return self.set_time <= other.set_time
    def __ge__(self,other):
        return self.set_time >= other.set_time
    def revisit(self):
        global waiting_list
        waiting_list.append(self.url)
        waiting_list=collections.deque(sorted(waiting_list)) #按照了rank来排序  在同样的rank下先进先出

def heap2wl():
    cur_time=time.time()
    while (len(heap)>0 and heap[0].set_time<=cur_time):
        event_top=heappop(heap)
        event_top.revisit()

In [3]:
#爬取url中内容
link_wait_num=0
link_finish_num=0
s = requests.session()
s.keep_alive = False
url_kw={}
table=open('url_table.csv','w')
abandon=0
def gettitle(soup):
    try:
        return (soup.title.string)
    except:
        try:
            return (soup.find('h1').string)
        except:
            try:
                return (soup.find('h2').string)
            except:
                try:
                    return (soup.find('h3').string)
                except:
                    return ("")

def get_keywords(soup,num=60):
    content=soup.get_text().replace('\n',' ').strip()
    return list(HanLP.extractKeyword(content,num))

def sim(kw_list1,kw_list2):
    num_inter=0
    num_union=len(kw_list1)
    for w in kw_list2:
        if w in kw_list1:
            num_inter+=1
        else:
            num_union+=1
    return float(num_inter)/float(num_union)
        

def search_href(soup):
    global link_wait_num
    if link_wait_num>=max_num*2:
        return
    if soup is None:
        return
    for a in soup.find_all('a', href=True,recursive=False):
        if a.get_text(strip=True): 
            new_url=a['href']
            link=re.findall(mode,new_url)
            if(len(link)>0):
                templink=link[0].rstrip(r"/")
                md5_dec=int(md5_(templink),16)
                if md5s[md5_dec]==0: #md5 deduplicate
                    global waiting_list
                    waiting_list.append(url_class(templink))
                    waiting_list=collections.deque(sorted(waiting_list))
                    md5s[md5_dec]=1 #仅在新网页引入时判断了md5，所以起点网站md5要手动置为1
                    link_wait_num+=1
                    if link_wait_num>=max_num*2:
                        return

def url_spider(url):
    global link_finish_num
    if link_finish_num>=max_num:
        return
    try:
        driver.get(r'http://'+url)
    except:
        driver.execute_script('window.stop()')
    try:
        data = driver.page_source
        soup=BeautifulSoup(data)
    except:
        return
    if soup is None:
        return
    #print(soup)####debuging
    #do something on soup/collect info
    try:
        temp_kw=get_keywords(soup.body)
        ### de-mirror
        for k in url_kw.values():
            if sim(k,temp_kw)>0.8: #相似度过高视作镜像
                global abandon
                abandon=1
                print('abandoned: ',url)
                return
        ###
        url_kw[url]=temp_kw
    except:
        #print(soup.body)
        return
    md5_filename=str(md5_(url))
    w=open('htmls\\'+md5_filename+'.html','w',encoding='utf-8')
    w.write(str(soup))
    w.close()
    link_finish_num+=1
    print('file_saved: ',md5_filename,' ',url)
    global table
    table.write(md5_filename+","+url+','+str(baidu_find(url))+'\n')
    #then add new urls to wl
    search_href(soup.body)
    try:
        for child in soup.body.contents:
            try:
                search_href(child)
            except:
                1
            try:
                for grand_child in child.contents:
                    try:
                        search_href(grand_child)
                    except:
                        1
                    try:
                        for ggc in grand_child.contents:
                            try:
                                search_href(ggc)
                            except:
                                1
                            try:
                                for sc in ggc.contents:
                                    try:
                                        search_href(sc)
                                    except:
                                        1
                            except:
                                1
                    except:
                        1
            except:
                1
    except:
        1        
    
def wl_deal():
    heap2wl()
    global waiting_list
    while len(waiting_list)>0 :
        time.sleep(0.01)#象征性地限制流速hhh
        heap2wl()
        cur_url=waiting_list.popleft() #deque后入前出
        url_spider(cur_url.url)
        global link_finish_num
        if link_finish_num>=max_num:
            break
        #print(cur_url.url)###########################################debug
        global abandon
        if abandon==0:
            heappush(heap,update_event(url=cur_url.url))    
        else:
            abandon=0

def wl_add_first(url):
    global waiting_list
    waiting_list.append(url_class(url))
    waiting_list=collections.deque(sorted(waiting_list))
    global link_wait_num
    link_wait_num+=1
    md5_dec_first=int(md5_(url),16)
    md5s[md5_dec_first]=1 #仅在新网页引入时判断了md5，所以起点网站md5要手动置为1
    
def run(urls):
    for url in urls:
        wl_add_first(url)
    while True:
        wl_deal()
        if link_finish_num>=max_num:
            print('all caught at least once')
            global table
            table.close()
            break
        time.sleep(0.1)


In [4]:
run([r'www.elecfans.com',r'it.sohu.com'])

file_saved:  e425fb6   www.elecfans.com
file_saved:  39997ef   it.sohu.com
file_saved:  a18b26c   bbs.elecfans.com
file_saved:  c97bf4b   t.elecfans.com
file_saved:  6168b3a   z.elecfans.com
file_saved:  d02048a   webinar.elecfans.com
file_saved:  74b6498   event.elecfans.com
file_saved:  63c26ea   www.huaqiu.cn
file_saved:  a95a3f3   www.tianyancha.com
file_saved:  f346d45   ishare.iask.sina.com.cn
file_saved:  e0c6660   www.qudong.com
file_saved:  5d5f730   www.52rd.com
file_saved:  dd464d4   smt.hqchip.com
file_saved:  9ef33bc   www.chexun.com
file_saved:  d092d05   www.bjx.com.cn
file_saved:  d79fc37   www.cecb2b.com
file_saved:  99602b7   www.eepw.com.cn
file_saved:  32eafc5   www.cnmo.com
file_saved:  d63f0e2   diy.elecfans.com
file_saved:  7222504   www.dianyuan.com
file_saved:  cdd3128   www.hqps.com
file_saved:  2321ff9   www.gongchang.com
file_saved:  be11d76   www.114ic.com
file_saved:  f8a5aeb   www.cnledw.com
file_saved:  44a489e   www.ic37.com
file_saved:  2df1dfe   www.d

In [5]:
print(time.time()-begin_t)

233.68538570404053


In [6]:
link_finish_num

60